In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import db_connection as db

In [7]:

# load your cleaned flight log
df = db.read_db("flights_cleaned")
print(df.columns)
df.info()
df.head()

Index(['latitude', 'longitude', 'gps_altitude_m', 'distance_m', 'speed_km/s',
       'climb_m', 'climb_m(delta)', 'climb_rate_m/s', 'glide_ratio', 'bearing',
       'delta_bearing', 'elapsed_time', 'delta_time', 'temp', 'pressure',
       'humidity', 'dew_point', 'wind_speed', 'wind_deg'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376485 entries, 0 to 376484
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   latitude        376485 non-null  float64
 1   longitude       376485 non-null  float64
 2   gps_altitude_m  376485 non-null  int64  
 3   distance_m      376485 non-null  float64
 4   speed_km/s      376485 non-null  float64
 5   climb_m         376485 non-null  float64
 6   climb_m(delta)  376485 non-null  float64
 7   climb_rate_m/s  376485 non-null  float64
 8   glide_ratio     376485 non-null  float64
 9   bearing         376485 non-null  int64  
 10  delta_bearing   376

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,36.980933,29.315317,2077,9.485190,34.146683,3.0,-12.0,3.0,3.161730,141,6.0,29.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
1,36.980883,29.315350,2077,6.292672,22.653618,3.0,-9.0,3.0,2.097557,151,10.0,30.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
2,36.980833,29.315383,2078,6.292672,22.653621,4.0,-5.0,4.0,1.573168,151,0.0,31.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
3,36.980767,29.315433,2080,8.634538,31.084336,3.0,-2.0,3.0,2.878179,149,2.0,32.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0
4,36.980700,29.315483,2082,8.634540,31.084343,2.0,1.0,2.0,4.317270,149,0.0,33.0,1.0,25.62,1008.0,35.0,9.04,0.66,75.0


In [9]:
feature_cols = [
    'gps_altitude_m', 'distance_m', 'speed_km/s', #'climb_m',
    'glide_ratio', 'bearing', 'delta_bearing',
    'temp', 'pressure', 'humidity', 'dew_point',
    'wind_speed', 'wind_deg'
]
X = df[feature_cols].values
y = df['climb_rate_m/s'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, shuffle=False
)

In [ ]:
# 5.1 Create sliding windows of length T
def create_sequences(X, y, T=10):
    Xs, ys = [], []
    for i in range(len(X) - T):
        Xs.append(X[i:i+T])
        ys.append(y[i+T])
    return np.array(Xs), np.array(ys)

T = 5  # e.g., 5 previous timesteps
X_seq, y_seq = create_sequences(X_scaled, y, T)


In [ ]:

# 5.2 Split
split = int(0.8 * len(X_seq))
X_tr, X_val = X_seq[:split], X_seq[split:]
y_tr, y_val = y_seq[:split], y_seq[split:]

# 5.3 Build LSTM
model = Sequential([
    LSTM(64, input_shape=(T, X_scaled.shape[1]), return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

es = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[es]
)

# 5.4 Evaluate
mse, mae = model.evaluate(X_val, y_val)
print("LSTM MAE:", mae)
print("LSTM RMSE:", np.sqrt(mse))

In [ ]:
# 1. Generate predictions
y_pred = model.predict(X_val).flatten()
actual = y_val

# 2. Compute residuals
residuals = actual - y_pred

# 3. Scatter: actual vs. predicted
plt.figure()
plt.scatter(actual, y_pred)
plt.xlabel("Actual climb rate (m/s)")
plt.ylabel("Predicted climb rate (m/s)")
plt.title("Actual vs Predicted Climb Rate")
plt.show()

# 4. Histogram of residuals
plt.figure()
plt.hist(residuals, bins=50)
plt.xlabel("Residual (Actual − Predicted) (m/s)")
plt.ylabel("Frequency")
plt.title("Histogram of Residuals")
plt.show()

In [ ]:
# residuals = actual - y_pred from before
res_mean = np.mean(residuals)
res_std  = np.std(residuals)
print(f"Mean residual: {res_mean:.3f} m/s")
print(f"Residual STD : {res_std:.3f} m/s")

for thr in (0.5, 1.0, 2.0):
    pct = (np.abs(residuals) <= thr).mean() * 100
    print(f"% of errors within ±{thr:.1f} m/s: {pct:.1f}%")

In [ ]:
mn = min(actual.min(), y_pred.min())
mx = max(actual.max(), y_pred.max())

plt.figure()
plt.scatter(actual, y_pred, alpha=0.3)
plt.plot([mn, mx], [mn, mx], 'r--', label='Ideal (y=x)')
plt.xlabel("Actual climb rate (m/s)")
plt.ylabel("Predicted climb rate (m/s)")
plt.title("Actual vs Predicted Climb Rate")
plt.legend()
plt.show()

In [ ]:
residuals = actual - y_pred  # Calculate prediction errors

df = pd.DataFrame({'Actual': actual, 'Predicted': y_pred})

plt.figure(figsize=(8, 6))
sns.histplot(data=df, x='Actual', y='Predicted', bins=50, pthresh=.1, cmap="mako")
plt.plot([df.Actual.min(), df.Actual.max()],
         [df.Actual.min(), df.Actual.max()],
         'r--', label='Ideal (y = x)')
plt.xlabel("Actual climb rate (m/s)")
plt.ylabel("Predicted climb rate (m/s)")
plt.title("Heatmap of Actual vs Predicted Climb Rate")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure()
plt.scatter(y_pred, residuals, alpha=0.3)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel("Predicted climb rate (m/s)")
plt.ylabel("Residual (Actual − Predicted) (m/s)")
plt.title("Residuals vs. Predicted")
plt.show()

In [ ]:
residuals = actual - y_pred  # from before

print("Mean residual:", np.mean(residuals))
print("Residual std:",   np.std(residuals))

for thr in (0.5, 1.0, 2.0):
    pct = (np.abs(residuals) <= thr).mean() * 100
    print(f"% of errors within ±{thr:.1f} m/s:", f"{pct:.1f}%")

In [ ]:
# model.save("lstm_model.h5")
model.save('lstm_model.keras')